In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# training data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/', one_hot=True)
trX, trY = mnist.train.images, mnist.train.labels
teX, teY = mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28, 1)  # 28*28*1 color img
teX = teX.reshape(-1, 28, 28, 1)  # -1 = n

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
# placeholders
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])
dropout_conv = tf.placeholder("float")
dropout_fc = tf.placeholder("float")

In [4]:
# weights
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))    # 3*3*1 Convolution, 32 feature maps
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
W4 = tf.Variable(tf.random_normal([128*4*4, 625], stddev=0.01))    # FC 2048 inputs, 625 outputs
W5 = tf.Variable(tf.random_normal([625, 10], stddev=0.01))    # 10 labels

In [5]:
# model (cudnn >= 5.1)
_L1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME'))    # shape=(?, 28, 28, 32)
L1 = tf.nn.max_pool(_L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')    # shape=(?, 14, 14, 32)
L1 = tf.nn.dropout(L1, dropout_conv)
_L2 = tf.nn.relu(tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME'))    # shape=(?, 14, 14, 64)
L2 = tf.nn.max_pool(_L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')    # shape=(?, 7, 7, 64)
L2 = tf.nn.dropout(L2, dropout_conv)
_L3 = tf.nn.relu(tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME'))    # shape=(?, 7, 7, 128)
L3 = tf.nn.max_pool(_L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')    # shape=(?, 4, 4, 128)
L3 = tf.reshape(L3, [-1, W4.get_shape().as_list()[0]])    # shape=(?, 2048)
L3 = tf.nn.dropout(L3, dropout_conv)
L4 = tf.nn.relu(tf.matmul(L3, W4))    # shape=(?, 625)
L4 = tf.nn.dropout(L4, dropout_fc)
hypothesis = tf.matmul(L4, W5)    # shape=(?, 10)
predict = tf.argmax(hypothesis, 1)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis, Y))
train = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

In [7]:
batch_size = 128
test_size = 256
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for i in range(10):
        training_batch = zip(range(0, len(trX), batch_size), range(batch_size, len(trX)+1, batch_size))
        for start, end in training_batch:
            sess.run(train, {X: trX[start:end], Y: trY[start:end], dropout_conv: 0.8, dropout_fc: 0.5})
        test_indices = np.arange(len(teX)) # get test batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        print(i, np.mean(np.argmax(teY[test_indices], axis=1) == sess.run(predict, {X: teX[test_indices], dropout_conv: 1.0, dropout_fc: 1.0})))

(0, 0.96484375)
(1, 0.9765625)
(2, 1.0)
(3, 0.9921875)
(4, 0.99609375)
(5, 0.99609375)
(6, 1.0)
(7, 0.99609375)
(8, 1.0)
(9, 0.98828125)
